In [122]:
import math

MAX_BITS = 8
BITS_SIZE = 2 ** MAX_BITS
BITS_MASK = BITS_SIZE - 1
MAX_PERMS = math.factorial(MAX_BITS)
RSA_BITS = 8
RSA_OUTPUT_BYTES = RSA_BITS*2 // 8

def bit_sub_encrypt(input: int, key: int) -> int:
    key %= MAX_PERMS
    output = 0
    used_bits = 0
    for old_bit_pos in range(MAX_BITS):
        choice_count = MAX_BITS - old_bit_pos
        zeroes_to_bit_pos = key // math.factorial(choice_count - 1) % choice_count
        cursor = 0
        while (bit_value := used_bits >> cursor & 1) == 1 or zeroes_to_bit_pos > 0:
            cursor += 1
            zeroes_to_bit_pos -= ~bit_value & 1
        output |= (input >> old_bit_pos & 1) << cursor
        used_bits |= 1 << cursor
    return output

def bit_sub_decrypt(input: int, key: int) -> int:
    key %= MAX_PERMS
    output = 0
    used_bits = 0
    for old_bit_pos in range(MAX_BITS):
        choice_count = MAX_BITS - old_bit_pos
        zeroes_to_bit_pos = key // math.factorial(choice_count - 1) % choice_count
        cursor = 0
        while (bit_value := used_bits >> cursor & 1) == 1 or zeroes_to_bit_pos > 0:
            cursor += 1
            zeroes_to_bit_pos -= ~bit_value & 1
        output |= (input >> cursor & 1) << old_bit_pos
        used_bits |= 1 << cursor
    return output

def bit_shift(input: int, sh: int) -> int:
    return (input >> sh % MAX_BITS | input << MAX_BITS - sh % MAX_BITS) & (2 ** MAX_BITS - 1)

def vigenere_encrypt(input: int, key: bytes, i: int) -> int:
    if len(key) == 0:
        return input
    return (input + key[i % len(key)]) % BITS_SIZE

def vigenere_decrypt(input: int, key: bytes, i: int) -> int:
    if len(key) == 0:
        return input
    return (input - key[i % len(key)]) % BITS_SIZE

def vernam_cipher(input: int, key: bytes, i: int) -> int:
    if len(key) == 0:
        return input
    return input ^ key[i % len(key)]

In [125]:
import random
import base64
from dataclasses import dataclass

@dataclass
class CipherKey:
    bit_cipher_key: int # 0 - 8!-1
    shift_init: int # 0-7
    shift_rate: int # 0-7
    vigenere_key: bytes # 0-255[<255]
    vernam_key: bytes # 0-255[<255]
    n: int = 0 # 16!-1

    def __post_init__(self):
        if not (0 <= self.bit_cipher_key < MAX_PERMS):
            raise ValueError("bit_cipher_key is out of range")
        if not (0 <= self.shift_init < MAX_BITS):
            raise ValueError("shift_init is out of range")
        if not (0 <= self.shift_rate < MAX_BITS):
            raise ValueError("shift_rate is out of range")
        if len(self.vigenere_key) > 255:
            raise ValueError("vigenere_key is out of range")
        if len(self.vernam_key) > 255:
            raise ValueError("vernam_key is out of range")
    
    def to_bytes(self) -> bytes:
        return self.bit_cipher_key.to_bytes(2, byteorder='big') \
            + bytes([self.shift_init, self.shift_rate, len(self.vigenere_key)]) \
            + self.vigenere_key \
            + bytes([len(self.vernam_key)]) \
            + self.vernam_key
    
    def to_base64(self) -> str:
        return base64.b64encode(self.to_bytes()).decode('utf-8')

    @classmethod
    def from_random(cls):
        return cls(
            bit_cipher_key = random.randint(0, MAX_PERMS - 1),
            shift_init = random.randint(0, MAX_BITS - 1),
            shift_rate = random.randint(0, MAX_BITS - 1),
            vigenere_key = bytes([random.randint(0, 255) for _ in range(random.randint(0, 255))]),
            vernam_key = bytes([random.randint(0, 255) for _ in range(random.randint(0, 255))])
        )

    """
    1                   : bit_cipher_key
    1                   : shift_init
    1                   : shift_rate
    1                   : vigenere_key_size
    |vigenere_key_size| : vigenere_key
    1                   : vernam_key_size
    |vernam_key_size|   : vernam_key
    """
    @classmethod
    def from_bytes(cls, b: bytes):
        def get_bytes(b_iter, size):
            return bytes([next(b_iter) for _ in range(size)])
        
        b_iter = iter(b)

        bit_cipher_key = int.from_bytes(get_bytes(b_iter, 2), byteorder='big')
        shift_init = next(b_iter) % MAX_BITS
        shift_rate = next(b_iter) % MAX_BITS
        vigenere_key = get_bytes(b_iter, next(b_iter))
        vernam_key = get_bytes(b_iter, next(b_iter))

        return cls(
            bit_cipher_key,
            shift_init,
            shift_rate,
            vigenere_key,
            vernam_key,
        )

    @classmethod
    def from_base64(cls, b64: str):
        return cls.from_bytes(base64.b64decode(b64))


class Cipher:
    def __init__(self, key: CipherKey = None) -> None:
        self.i = 0
        self.key = key
    
    def set_key(self, key: CipherKey) -> None:
        self.key = key
    
    def reset(self) -> None:
        self.i = 0
    
    def encrypt(self, x: int) -> int:
        if self.key is None:
            raise Exception('Key is Omitted')
        x = bit_shift(x, self.key.shift_init + self.i * self.key.shift_rate)
        x = bit_sub_encrypt(x, self.key.bit_cipher_key)
        x = vigenere_encrypt(x, self.key.vigenere_key, self.i)
        x = vernam_cipher(x, self.key.vernam_key, self.i)
        self.i += 1
        return int(x)

    def decrypt(self, x: int) -> int:
        if self.key is None:
            raise Exception('Key is Omitted')
        x = vernam_cipher(x, self.key.vernam_key, self.i)
        x = vigenere_decrypt(x, self.key.vigenere_key, self.i)
        x = bit_sub_decrypt(x, self.key.bit_cipher_key)
        x = bit_shift(x, -self.key.shift_init - self.i * self.key.shift_rate)
        self.i += 1
        return x
# print(CipherKey(1, 2, 3, 'hello world'.encode(), 'hello thing'.encode()))

In [256]:
import math
import ipywidgets as widgets
import random
from IPython.display import display


# RSA ALGORITHM

output = widgets.Output()

# Fermat primality test
def is_prime(n):
    if n == 2 or n == 3:
        return True
    if n % 2 == 0:
        return False
    for _ in range(100):
        a = random.randint(2, n - 2)
        if pow(a, n - 1, n) != 1:
            return False
    return True

# Generate a prime number for RSA
def generate_prime(n):
    while True:
        p = random.randint(2**(n-1), 2**n)
        if is_prime(p):
            return p


# Generate p and q
p = generate_prime(8)
q = None
while True:
    q = generate_prime(8)
    if q != p:
        break

# Calculate n
n = p * q

# Calculate phi(n)
phi_n = (p - 1) * (q - 1)


def generate_e(phi_n):
    while True:
        e = random.randint(2, phi_n - 1)
        if math.gcd(e, phi_n) == 1:
            return e

def generate_d(e, phi_n):
    def extended_gcd(a, b):
        if b == 0:
            return a, 1, 0
        gcd, x, y = extended_gcd(b, a % b)
        return gcd, y, x - (a // b) * y

    _, d, _ = extended_gcd(e, phi_n)
    d %= phi_n
    return d

e = generate_e(phi_n)
d = generate_d(e, phi_n)

print(p, q, n, phi_n, e, d)

input = 121
encrypted = pow(input, e, n)
decrypted = pow(encrypted, d, n)
print(input, encrypted, decrypted)


239 181 43259 42840 20801 23081
121 7327 121


In [ ]:
import ipywidgets as widgets

output = widgets.Output()

@widgets.interact
def run(
    message = widgets.Textarea(value='Hello World', placeholder='Type something', description='String:', disabled=False), 
    cipher_key = (0, MAX_PERMS-1, 1), 
    shift_init = (0, MAX_BITS-1, 1),
    shift_rate = (0, MAX_BITS*2-1, 1),
    vigenere_key = "vigenere_key",
    vernam_key = "vernam_key"
):
    # ENCRYPTION
    encrypt = ""
    for i, char in enumerate(message):
        msg_num = ord(char)
        msg_num = bit_shift(msg_num, shift_init + i * shift_rate)
        msg_num = bit_sub_encrypt(msg_num, int(cipher_key))
        msg_num = vigenere_encrypt(msg_num, vigenere_key, i)
        msg_num = vernam_cipher(msg_num, vernam_key, i)
        encrypt += chr(msg_num)
    print(encrypt)
    print()
    # DECRYPTION
    decrypt = ""
    for i, char in enumerate(encrypt):
        msg_num = ord(char)
        msg_num = vernam_cipher(msg_num, vernam_key, i)
        msg_num = vigenere_decrypt(msg_num, vigenere_key, i)
        msg_num = bit_sub_decrypt(msg_num, int(cipher_key))
        msg_num = bit_shift(msg_num, -shift_init - i * shift_rate)
        decrypt += chr(msg_num)
    print(decrypt)

interactive(children=(Textarea(value='Hello World', description='String:', placeholder='Type something'), IntS…

In [132]:
import hashlib

input_filename = './Output/Test.txt'
encrypted_filename = './Output/Encrypted.txt'
decrypted_filename = './Output/Decrypted.txt'

key = CipherKey.from_random()

with open(input_filename, 'rb') as input_file:
    with open(encrypted_filename, 'wb') as output_file:
        cipher = Cipher(key)
        # copy by byte to encrypted_filename
        while (byte := input_file.read(1)):
            output_file.write(cipher.encrypt(int.from_bytes(byte, byteorder='big')).to_bytes(1, byteorder='big'))

with open(encrypted_filename, 'rb') as input_file:
    with open(decrypted_filename, 'wb') as output_file:
        cipher = Cipher(key)
        # copy by byte to decrypted_filename
        while (byte := input_file.read(1)):
            output_file.write(cipher.decrypt(int.from_bytes(byte, byteorder='big')).to_bytes(1, byteorder='big'))


def get_file_hash(filename):
    with open(filename, 'rb') as file:
        data = file.read()
        hash_object = hashlib.sha256(data)
        return hash_object.hexdigest()

input_hash = get_file_hash(input_filename)
encrypted_hash = get_file_hash(encrypted_filename)
decrypted_hash = get_file_hash(decrypted_filename)

print("Key:", key.to_base64())
print("Input File Hash:", input_hash)
print("Encrypted File Hash:", encrypted_hash)
print("Decrypted File Hash:", decrypted_hash)


Key: ImkEB6Lv80T0eO9yVaaXPi8VBvdD73Xgu0kScVF4UmVk0nqJp57RCdxGJWRFjAkJzlUM1aDwBLVbnO+xnYS3xwQCgZLMfjBmReEtO9BMcTRVvyJ0+tLRJaI/Zdy8YSbb+9QfqIMDN77VKvhqsM5GkoqgsQJH4pBS+xpIWUFo4XxAigmTnsylZh7Iv4lLQdJIHriPZ5TTbIR++EB8YE6ErijUItQwvGZRYa1cUTtEepAewhfWpzZlqIvuMSr4i8OSxfYzvqfNtomL0GRiVr4AJGImTQUy6fiXd9eHKjgAWuuucIevrQlzSYo1vohe/gdTN7Ix/F0MQ+jj
Input File Hash: ce0a7e27f6ba47cd29f69d5f44d54463bb0c5e9e8e7a1750dd2bfc23bae951ce
Encrypted File Hash: 776cc5f2c8db197e3de7be3f71ca1d35fa0a4a2ac617adf9c4e8aeee8deeeae6
Decrypted File Hash: ce0a7e27f6ba47cd29f69d5f44d54463bb0c5e9e8e7a1750dd2bfc23bae951ce
